# VacationPy
---

Starter Code to Import Libraries and Load the Weather and Coordinates Data

Adapted by Molly Ingram

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests

# Import API key
from config import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,walvis bay,-22.9575,14.5053,22.53,78,98,0.51,NaN,2024-02-06
1,1,olonkinbyen,70.9221,-8.7187,-1.24,89,76,8.55,SJ,2024-02-06
2,2,bethel,41.3712,-73.4140,5.11,50,0,2.57,US,2024-02-06
3,3,saint-francois,46.4154,3.9054,7.58,88,100,2.57,FR,2024-02-06
4,4,grytviken,-54.2811,-36.5092,13.01,42,75,5.19,GS,2024-02-06


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
# I'd like to go somewhere with highs between 55 and 75 F (roughly 13-23 C)
filtered_city_data = city_data_df.loc[(city_data_df['Max Temp']<=23) & (city_data_df['Max Temp']>=13)]

#Fixing Namibia country code
filtered_city_data.loc[filtered_city_data['Country'].isna(), 'Country'] = 'NA'

# Drop any rows with null values
np.where(pd.isnull(filtered_city_data))
np.where(pd.isna(filtered_city_data))
    #We have no missing or null values


# Display sample data
filtered_city_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,walvis bay,-22.9575,14.5053,22.53,78,98,0.51,NA,2024-02-06
4,4,grytviken,-54.2811,-36.5092,13.01,42,75,5.19,GS,2024-02-06
7,7,tokar,18.4254,37.7290,22.12,56,7,4.96,SD,2024-02-06
12,12,waitangi,-43.9535,-176.5597,17.68,94,62,2.24,NZ,2024-02-06
13,13,puerto natales,-51.7236,-72.4875,22.25,28,75,6.17,CL,2024-02-06


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data[['City', 'Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,walvis bay,NA,-22.9575,14.5053,78,
4,grytviken,GS,-54.2811,-36.5092,42,
7,tokar,SD,18.4254,37.7290,56,
12,waitangi,NZ,-43.9535,-176.5597,94,
13,puerto natales,CL,-51.7236,-72.4875,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
limit = 1
radius = 5000 #hotels within 5km of the coordinates
params = {
   "limit":limit,
   "categories": f"accommodation.hotel",
   "apiKey":geoapify_key
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
walvis bay - nearest hotel: Atlantic
grytviken - nearest hotel: No hotel found
tokar - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
puerto natales - nearest hotel: Dorotea Patagonia Hostel
ulladulla - nearest hotel: Hotel Marlin
mitu - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
yuma - nearest hotel: Hilton Garden Inn Yuma Pivot Point
mhamid - nearest hotel: Hotel Kasbah Azalay
margaret river - nearest hotel: Margaret River Hotel
kailua-kona - nearest hotel: Kona Seaside Hotel
blackmans bay - nearest hotel: Villa Howden
broken hill - nearest hotel: Ibis Styles
ancud - nearest hotel: Cabañas
coquimbo - nearest hotel: Hotel Iberia
nova sintra - nearest hotel: Residência Ka Dencho
ulysses - nearest hotel: Single Tree Inn Hotel O
ribeira grande - nearest hotel: No hotel found
richards bay - nearest hotel: SUN1 Richards Bay
kerikeri - nearest hotel: Avalon Resort
hawaiian paradise park - nearest hotel: No hotel fo

,City,Country,Lat,Lng,Humidity,Hotel Name
0,walvis bay,NA,-22.9575,14.5053,78,Atlantic
4,grytviken,GS,-54.2811,-36.5092,42,No hotel found
7,tokar,SD,18.4254,37.7290,56,No hotel found
12,waitangi,NZ,-43.9535,-176.5597,94,Hotel Chathams
13,puerto natales,CL,-51.7236,-72.4875,28,Dorotea Patagonia Hostel
...,...,...,...,...,...,...
538,dhari,IN,21.3333,71.0167,46,No hotel found
544,senanga,ZM,-16.1167,23.2667,85,Senanga Safari Lodge
545,netley,GB,50.8763,-1.3540,90,Southampton Harbour Hotel and Spa
551,vredendal,ZA,-31.6683,18.5012,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)